<a href="https://colab.research.google.com/github/wisrovi/TFM2022/blob/main/TFM2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Autor

Autor: William S. Rodriguez V.

Link colab: https://colab.research.google.com/drive/1KSSPwElIl1MZnMRn0XdXlJPlcGh-i1ST#scrollTo=dv_hLoOBFj33

Github: https://github.com/wisrovi/TFM2022

In [ ]:
#@title Autor

%%html



<!--  https://ghlinkcard.com/  -->
<a href="https://github.com/wisrovi/TFM2022">
<img src="https://github-link-card.s3.ap-northeast-1.amazonaws.com/wisrovi/TFM2022.png" width="460px">
</a> 
<br/>
<a href="https://profile-summary-for-github.com/user/wisrovi">Estadisticas Repo</a>

<br/>
<br/>
<br/>
<br/>
<br/>

<script src="https://platform.linkedin.com/badges/js/profile.js" 
async defer type="text/javascript"></script>
<div class="badge-base LI-profile-badge" data-locale="es_ES" data-size="large" 
data-theme="dark" data-type="HORIZONTAL" data-vanity="wisrovi-rodriguez" 
data-version="v1"><a class="badge-base__link LI-simple-link" 
href="https://co.linkedin.com/in/wisrovi-rodriguez?trk=profile-badge">William Rodriguez</a></div>


<br/>
<br/>
<br/>
<br/>
<br/>


<iframe src="https://kaggle-card.chienhsiang-hung.eu.org/api/basic?wisrovi" 
width="62%" height="280" style="border:none; min-width: 80px;"></iframe>




# 0. Config

In [ ]:
#@title basic config
FOLDER_OUT = "/gdrive/MyDrive/TFM/"
FOLDER_INT = "musicnet/musicnet"

USE_COLAB = True

In [ ]:
#@title CONFIG
#@markdown Configuracion del sistema.
#@markdown ---
#@markdown ---

porcentaje_PCA = 93  #@param {type: "slider", min: 80, max: 98}
tiempo_split = 2  #@param {type: "slider", min: 1, max: 12}
porcentaje_split_train_valid = 20  #@param {type: "slider", min: 10, max: 35}
#@markdown ---
procesar_data_kaggle = False #@param {type: "boolean"} 
aplicar_data_incrementation = True #@param {type: "boolean"} 
refit = True #@param {type: "boolean"} 
#@markdown ---


In [ ]:
#@title CONFIG

MINIMA_VARIANA_EXPLICADA = 0.93 if not USE_COLAB else porcentaje_PCA/100 # se debe definir porque este valor
TIME_SPLIT = 1 if not USE_COLAB else tiempo_split # falta definir porque este split de 2

# Folder
FOLDER_SAVE_NORMALIZADOR_PCA = FOLDER_OUT + f'normalizador_pca_{str(TIME_SPLIT)}seg.pkl'
FOLDER_MODEL = FOLDER_OUT + f"randomforest_{str(TIME_SPLIT)}seg.pkl"

FOLDER_TRAIN_DATA = FOLDER_OUT + f"train_data_{str(TIME_SPLIT)}seg.csv"
FOLDER_TRAIN_LABEL = FOLDER_OUT + f"train_label_{str(TIME_SPLIT)}seg.csv"
FOLDER_TEST_DATA = FOLDER_OUT + f"test_data_{str(TIME_SPLIT)}seg.csv"
FOLDER_TEST_LABEL = FOLDER_OUT + f"test_label_{str(TIME_SPLIT)}seg.csv"

FOLDER_LABEL = FOLDER_OUT + "all_label.csv"

# Model
SPLIT_DATA_TRAIN = 0.2 if not USE_COLAB else porcentaje_split_train_valid/100 

EXTRACT_DATA_AUDIOS = False if not USE_COLAB else procesar_data_kaggle 
FIND_DATA_INCREMENT = True if not USE_COLAB else aplicar_data_incrementation 
REFIT = True if not USE_COLAB else refit 

# 1. Obtener la llave de kaggle

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [ ]:
if EXTRACT_DATA_AUDIOS:
    # Creamos carpeta oculta en ambiente de linux sobre colab
    !mkdir ~/.kaggle

    # Copiar el archivo JSON  a la carpeta oculta que creamos
    !cp '/gdrive/MyDrive/TFM/kaggle.json' ~/.kaggle/

    # Cambiamos los permisos para permitir lectura de las credenciales
    !chmod 600 ~/.kaggle/kaggle.json

    pass

# 2. Instalando librerias faltantes

In [ ]:
!pip install -q kaggle
!pip install pyspark
!pip install nlpaug===1.1.11

# 3. Importando recursos

In [ ]:
#@title validando librosa
try:
    import librosa
except:
    # !pip uninstall --yes librosa
    !pip install librosa===0.7.2 --force-reinstall
import librosa
print("current vers", librosa.__version__)

current vers 0.8.1


In [ ]:
#@title librerias para preprocesamiento de datos
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

import numpy as np
import pickle

In [ ]:
#@title librerias para modelado y entrenamiento
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

import sklearn.metrics as metrics


In [ ]:
#@title librerias para bigdata
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import *

In [ ]:
#@title libreria para split de audio
import scipy.io.wavfile as wav
import pandas as pd


class Corte_audio():
    SEGUNDOS_CORTE = 5

    def __init__(self, config_time: int = 5, save_audios: str = None):
        self.SEGUNDOS_CORTE = config_time
        self.save_audios = save_audios

    def __hallar_instrumentos(self, corte_start, corte_end, instrument, start_time):
        acumulador = list()
        for (i, time_start) in enumerate(start_time):
            if corte_end >= start_time[i] >= corte_start:
                # print(start_time[i], end_time[i], instrument[i])
                if not instrument[i] in acumulador:
                    acumulador.append(instrument[i])
            else:
                if time_start > corte_end:
                    break
        return acumulador

    def ___save_data(self, data_save, file):
        df = pd.DataFrame(data=data_save, columns=['instrument'], index=None, dtype=None, copy=False)
        df.to_csv(file, index=False)

    def split_data(self, path_wav: str = "", path_csv: str = "", tag: str = "dat"):
        (rate, sig) = wav.read(path_wav)
        labels = pd.read_csv(path_csv)

        instrument = labels['instrument']
        start_time = labels['start_time']

        pivote = 0
        corte = rate * self.SEGUNDOS_CORTE

        muestras = list()
        instrumentos = list()
        for _ in range(round(len(sig) / corte)):
            corte_start, corte_end = pivote, pivote + corte
            data = sig[corte_start:corte_end]
            muestras.append(data)
            instrumentos.append(self.__hallar_instrumentos(corte_start, corte_end, instrument, start_time))
            pivote += corte

        if pivote < len(sig):
            data = sig[pivote:]
            muestras.append(data)
            instrumentos.append(self.__hallar_instrumentos(pivote, len(sig), instrument, start_time))

        if self.save_audios is not None:
            for i, data in enumerate(muestras):
                wav.write(self.save_audios + f"{tag}_{i}.wav", rate, data)
                self.___save_data(instrumentos[i], self.save_audios + f"{tag}_{i}.csv")

        return muestras, instrumentos, rate

In [ ]:
#@title libreria para extraccion de caracteristicas
import matplotlib.pyplot as plt
import librosa.display
import numpy as np


class ProcessAudio(object):
    data = None

    def __init__(self, sr:int = 44100):
        self.mfcc = None
        self.zcr = None
        self.rolloff = None
        self.spec_bw = None
        self.spec_cent = None
        self.rmse = None
        self.chroma_stft = None
        self.sr = sr

    def set_data(self, data):
        self.data = data

    def display_waveform(self):
        if self.data is None:
            return None
        # display waveform
        plt.figure(figsize=(14, 5))
        librosa.display.waveplot(self.data , sr=self.sr)

    def get_croma(self):
        if self.data is None:
            return None
        self.chroma_stft = librosa.feature.chroma_stft(y=self.data, sr=self.sr)
        return self.chroma_stft

    def get_rmse(self):
        if self.data is None:
            return None
        self.rmse = librosa.feature.rms(y=self.data)
        return self.rmse

    def get_centroide_espectral(self):
        """centroide espectral"""
        if self.data is None:
            return None
        self.spec_cent = librosa.feature.spectral_centroid(y=self.data, sr=self.sr)
        return self.spec_cent

    def get_ancho_banda_espectral(self):
        if self.data is None:
            return None
        self.spec_bw = librosa.feature.spectral_bandwidth(y=self.data, sr=self.sr)
        return self.spec_bw

    def get_rolloff(self):
        """tambien conocido como reduccion espectral"""
        if self.data is None:
            return None
        self.rolloff = librosa.feature.spectral_rolloff(y=self.data, sr=self.sr)
        return self.rolloff

    def get_cruce_por_cero(self):
        if self.data is None:
            return None
        self.zcr = librosa.feature.zero_crossing_rate(self.data)
        return self.zcr

    def get_mfcc(self):
        if self.data is None:
            return None
        self.mfcc = librosa.feature.mfcc(y=self.data, sr=self.sr)
        return self.mfcc

    def get_all(self, i:int) -> list:
        if self.data is None:
            return []

        self.get_croma()
        self.get_rmse()
        self.get_centroide_espectral()
        self.get_ancho_banda_espectral()
        self.get_rolloff()
        self.get_cruce_por_cero()
        self.get_mfcc()

        data_compresed = f'train{i} {np.mean(self.chroma_stft)} {np.mean(self.rmse)} {np.mean(self.spec_cent)} {np.mean(self.spec_bw)} {np.mean(self.rolloff)} {np.mean(self.zcr)}'
        for e in self.mfcc:
            data_compresed += f' {np.mean(e)}'

        return data_compresed.split()

In [ ]:
#@title libreria para aumento de datos de audio

import nlpaug.augmenter.audio as naa
import librosa.display as librosa_display


class AudioAugmentation:
    graph: bool = False
    print_middleware: bool = False

    def __init__(self, audio_file, graph=False, save: str = None):
        self.audio_file = audio_file
        self.graph = graph
        self.save = save
        if isinstance(audio_file, str):
            print("Cargando archivo de audio")
            self.name_file = audio_file.split(".")[-2].split("/")[-1]
            self.data, self.rate = self.__read_audio_file(self.audio_file)
        elif isinstance(audio_file, tuple):
            #print("Audio file is a tuple")
            self.data, self.rate, self.name_file = audio_file
            #print(len(self.data), self.rate, self.name_file)

    def middleware_in(atributo1: str = "", ):
        def _middleware_in(f):
            def wrapper(self, *args, **kwargs):
                # ejecucion antes de ejecutar la funcion
                if self.print_middleware:
                    print(f"Middleware in {f.__name__} ({atributo1})")

                # ejecucion de la funcion
                ejecucion_ok = True
                try:
                    salida = f(self, *args, **kwargs)
                except Exception as e:
                    ejecucion_ok = False
                    salida = e
                    print(f"Error en la ejecucion de la funcion {f.__name__}: {e}")

                # ejecucion despues de ejecutar la funcion
                if ejecucion_ok:
                    if self.graph:
                        self.plot_audio(salida)

                    if self.save is not None:
                        try:
                            if isinstance(salida, list):
                                if self.print_middleware:
                                    print(salida)
                                salida = np.array(salida)
                            self.write_audio_file(self.save + self.name_file + "_" + f"{f.__name__}.wav", salida, self.rate)
                        except Exception as e:
                            if self.print_middleware:
                                print(f"Error al guardar el archivo: {e}")
                            print(type(salida))

                return salida

            return wrapper

        return _middleware_in

    def plot_audio(self, data2=None):
        librosa_display.waveshow(self.data, sr=self.rate, alpha=0.5)
        if data2 is not None:
            librosa_display.waveshow(data2, sr=self.rate, color='r', alpha=0.25)

        plt.tight_layout()
        plt.show()

    @middleware_in(atributo1="loudness_f")
    def loudness(self):
        aug = naa.LoudnessAug()
        augmented_data = aug.augment(self.data)[0]
        return augmented_data

    @middleware_in(atributo1="mask_f")
    def add_mask(self):
        aug = naa.MaskAug(sampling_rate=self.rate, mask_with_noise=False)
        augmented_data = aug.augment(self.data)[0]
        return augmented_data

    @middleware_in(atributo1="pitch_f")
    def pitch(self, fact=(2,3)):
        aug = naa.PitchAug(sampling_rate=self.rate, factor=fact)
        augmented_data = aug.augment(self.data)[0]
        return augmented_data

    @middleware_in(atributo1="original")
    def get_original(self):
        return self.data

    @middleware_in(atributo1="crop")
    def add_crop(self, porcentaje: float = 0.5):
        crop = naa.CropAug(sampling_rate=self.rate)
        augmented_data = crop.augment(self.data)[0]
        return augmented_data

    @staticmethod
    def __read_audio_file(file_path):
        input_length = 16000
        data, rate = librosa.core.load(file_path)
        if len(data) > input_length:
            data = data[:input_length]
        else:
            data = np.pad(data, (0, max(0, input_length - len(data))), "constant")
        return data, rate

    @staticmethod
    def write_audio_file(file, data, sample_rate):
        write(file, sample_rate, data)

    @staticmethod
    def plot_time_series(data):
        fig = plt.figure(figsize=(14, 8))
        plt.title('Raw wave ')
        plt.ylabel('Amplitude')
        plt.plot(np.linspace(0, 1, len(data)), data)
        plt.show()

    @middleware_in(atributo1="ruido")
    def add_noise(self, factor_ruido=0.005):
        noise = np.random.randn(len(self.data))
        data_noise = self.data + factor_ruido * noise
        data_noise = data_noise.astype(type(self.data[0]))  # Cast back to same
        return data_noise

    @middleware_in(atributo1="ruido2")
    def add_noise2(self):
        aug = naa.NoiseAug()
        augmented_data = aug.augment(self.data)[0]
        return augmented_data

    @middleware_in(atributo1="shift")
    def shift(self):
        return np.roll(self.data, 1600)

    @middleware_in(atributo1="stretch")
    def stretch(self, rate_stretch=1.0):
        input_length = 16000
        data = librosa.effects.time_stretch(self.data, rate=rate_stretch)
        if len(data) > input_length:
            data = data[:input_length]
        else:
            data = np.pad(data, (0, max(0, input_length - len(data))), "constant")
        return data

    @middleware_in(atributo1="speed")
    def speed(self):
        aug = naa.SpeedAug()
        augmented_data = aug.augment(self.data)[0]
        return augmented_data

    @middleware_in(atributo1="normalizador")
    def normalizer(self):
        aug = naa.NormalizeAug(method='minmax')
        augmented_data = aug.augment(self.data)[0]
        return augmented_data

    @middleware_in(atributo1="polarizador")
    def polarizer(self):
        aug = naa.PolarityInverseAug()
        augmented_data = aug.augment(self.data)[0]
        return augmented_data

In [ ]:
#@title libreria para cargar los audios, preprocesarlos y montarlos en matrix numpy
class Audio_K(AudioAugmentation):

    def __init__(self, file_path, label: list, save: str = None, grafica: bool = False):
        super().__init__(file_path, save=save, graph=grafica)
        self.label = label

    def aumentar(self):
        all_data = [
            self.get_original(),
            self.add_noise(factor_ruido=0.05),
            self.add_noise2(),
            self.stretch(rate_stretch=0.8),
            self.shift(),
            self.add_crop(),
            self.loudness(),
            self.speed(),
            self.normalizer(),
            self.polarizer()
        ]

        all_label = [self.label for _ in range(len(all_data))]
        return all_data, all_label

In [ ]:
#@title funcion para conteo tiempo de ejecucion de una funcion
from functools import wraps
from time import time


def count_elapsed_time(f):
    @wraps(f)
    def cronometro(*args, **kwargs):
        t_inicial = time()  # tomo la hora antes de ejecutar la funcion
        salida = f(*args, **kwargs)
        t_final = time()  # tomo la hora despues de ejecutar la funcion
        print('Tiempo transcurrido (en segundos): {}'.format(t_final - t_inicial))
        return salida

    return cronometro

In [ ]:
#@title evitar que el cuaderno muestre alertas innecesarias
import warnings

warnings.filterwarnings('ignore')

# 4. Clonando los datos

In [ ]:
#@title descargo los datos de kaggle
if EXTRACT_DATA_AUDIOS:
    # Descargamos archivos indicando el usuario del propietario de los datos en kaggle y el nombre de dataset
    ! kaggle datasets download imsparsh/musicnet-dataset --force
else:
    print("No se clonan los datos de kaggle porque la opcion esta deshabilitada en la configuración")

No se clonan los datos de kaggle porque la opcion esta deshabilitada en la configuración


In [ ]:
#@title descomprimo los datos descargados de kaggle
import zipfile
import os

if EXTRACT_DATA_AUDIOS:
    # Descomprimir el archivo de kaggle
    for file in os.listdir():
        if file.endswith('.zip'):
            zip_ref = zipfile.ZipFile(file, 'r')
            zip_ref.extractall()
            zip_ref.close()

In [ ]:
#@title elimino el archivo original, pues ya no es necesario luego de descomprimir
if EXTRACT_DATA_AUDIOS:
    !rm musicnet-dataset.zip

# 5. Preprocesando los datos

In [ ]:
# Iniciar conexión de spark para analizar con bigdata los datos luego de preprocesarlos
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

# 5.1. pipeline y funciones para preprocesar los datos

In [ ]:
# iniciador de clases

pca_pipe = make_pipeline(StandardScaler(), PCA(MINIMA_VARIANA_EXPLICADA))

In [ ]:
if EXTRACT_DATA_AUDIOS:
    # organizando los datos en train y test (data y label para cada uno)

    TRAIN = {}
    TEST = {}

    @count_elapsed_time
    def crear_diccionarios_train_test():
        for dirname, _, filenames in os.walk(FOLDER_INT):
            for filename in filenames:
                file = os.path.join(dirname, filename)
                name_file, extension = filename.split(".")

                if extension == "csv" or extension == "wav":
                    if dirname.find("train")>0:
                        if name_file not in TRAIN:
                            TRAIN[name_file] = {}
                        if filename.find("csv")>0:
                            TRAIN[name_file]['label'] = file
                        else:
                            TRAIN[name_file]['data'] = file
                    else:
                        if name_file not in TEST:
                            TEST[name_file] = {}
                        if filename.find("csv")>0:
                            TEST[name_file]['label'] = file
                        else:
                            TEST[name_file]['data'] = file
    crear_diccionarios_train_test()

In [ ]:
def save_data(data_save, file):
    df = pd.DataFrame(data=data_save)
    df.to_csv(file)

def read_data(file, to_numpy: bool = False):
    df = spark.read.csv(file, 
                        header=True,
                        mode="DROPMALFORMED",)
    df_pandas = df.toPandas()
    df_pandas = df_pandas.drop('_c0', axis=1)
    if to_numpy:
        df_pandas = df_pandas.to_numpy()
    return df, df_pandas

In [ ]:
if EXTRACT_DATA_AUDIOS:
    TODOS_LABEL = {}
else:
    TODOS_LABEL = read_data(FOLDER_LABEL, to_numpy=True)[1]
    TODOS_LABEL = tuple([ d[0] for d in TODOS_LABEL.tolist()])
    print(TODOS_LABEL)

('1', '7', '41', '42', '43', '44', '61', '69', '71', '72', '74')


In [ ]:
@count_elapsed_time
def files_to_data(diccionario_datos, buscar_todos_label: bool = False, save_new_data: str = None, data_aumentation: bool = False):
    global TODOS_LABEL
    conteo = 0
    conteo2 = 0

    DATA = []
    LABEL = []

    rate = 0

    cortador = Corte_audio(config_time=TIME_SPLIT, save_audios=save_new_data)

    for name_file, value in diccionario_datos.items():
        if 'data' in value and 'label' in value:
            conteo += 1
            conteo2 += 1
            print(f"{name_file}({conteo2})", end=" - ")
            if conteo >= 20:
                print()
                conteo = 0

            muestras_wav, instrumentos, rate = cortador.split_data(value['data'], value['label'])

            if data_aumentation and buscar_todos_label:
                new_muestras = []
                new_instruments = []
                for id_audio, dat in enumerate(muestras_wav):
                    data_audio_para_amplificar= (dat, rate, "data_" + str(id_audio))
                    label_audio_para_amplificar = instrumentos[id_audio]
                    data_amplificada, label_amplificada = Audio_K(data_audio_para_amplificar, label=label_audio_para_amplificar).aumentar()
                    #print(len(data_amplificada))
                    for id_new_audio, new_audio in enumerate(data_amplificada):
                        new_data_add = new_audio
                        new_label_add = label_amplificada[id_new_audio]

                        ok = True
                        try:
                            cte = 110250
                            if (TIME_SPLIT*rate) > len(new_data_add):
                                new_data_add += [0 for _ in range(cte*TIME_SPLIT)]
                            if (TIME_SPLIT*rate) < len(new_data_add):
                                new_data_add = new_data_add[:TIME_SPLIT*cte]
                        except Exception as e:
                            ok = False

                        if ok:
                            new_muestras.append(new_data_add)
                            new_instruments.append(new_label_add)

                muestras_wav, instrumentos = new_muestras, new_instruments

            processAudio = ProcessAudio(rate)
            for id_audio, dat in enumerate(muestras_wav):
                processAudio.set_data(dat)
                # print(len(dat), end = ">")
                ok = True
                caracteristicas = None
                try:
                    caracteristicas = processAudio.get_all(id_audio)  # Extrayendo caracteristicas audios, salen 26 caracteristicas
                except:
                    ok = False
                if ok:
                    DATA.append(caracteristicas[1:])
                    LABEL.append(instrumentos[id_audio])

    # buscando todos los label
    if buscar_todos_label:
        for lab in LABEL:
            for la in lab:
                if la not in TODOS_LABEL:
                    TODOS_LABEL[la] = 0
        TODOS_LABEL = tuple(sorted(TODOS_LABEL))
        save_data(data_save=TODOS_LABEL, file=FOLDER_LABEL)

    # expandiendo los label a su respectivo vector de etiquetas
    for j, lab in enumerate(LABEL):
        new_label = [0 for _ in TODOS_LABEL]
        for la in lab:
            for i, l in enumerate(TODOS_LABEL):
                if l == la:
                    new_label[i] = 1
        LABEL[j] = new_label


    # convirtiendo a numpy los datos
    DATA = np.array(DATA, dtype=float)
    LABEL = np.array(LABEL, dtype=float)

    return DATA, LABEL

In [ ]:
mean = lambda lst: int((sum(lst) / len(lst)) * 100) / 100

def calcular_porcentajes_aciertos(y_f, y_t):
    verdaderos = dict()
    falsos = dict()
    for j in range(y_f.shape[1]):
        verdaderos[j] = 0
        falsos[j] = 0

    for i in range(y_f.shape[0]):
        for j in range(y_f.shape[1]):
            if y_f[i][j] == y_t[i][j]:
                verdaderos[j] += 1
            else:
                falsos[j] += 1

    for j in range(y_f.shape[1]):
        # y_final.shape[1] -> 100%
        # verdaderos[j]    -> X
        verdaderos[j] = int(verdaderos[j] * 100 / y_f.shape[0])
        falsos[j] = int(falsos[j] * 100 / y_f.shape[0])

    return verdaderos, falsos, str(mean([v for i, v in verdaderos.items()])) + "%"

In [ ]:
def preparar_datos_para_modelo(datos):
    normalizador_pca = pickle.load(open(FOLDER_SAVE_NORMALIZADOR_PCA, 'rb'))
    x_for_model = normalizador_pca.transform(X=datos)
    return x_for_model

# 5.2. Cargando data

In [ ]:
DATA, LABEL = None, None
if EXTRACT_DATA_AUDIOS:
    DATA, LABEL = files_to_data(TRAIN, 
                                buscar_todos_label=True, 
                                data_aumentation=FIND_DATA_INCREMENT)
else:
    print("No se leeran los archivos de audios para convertirlos a numpy porque la opcion esta deshabilitada en la configuración")

No se leeran los archivos de audios para convertirlos a numpy porque la opcion esta deshabilitada en la configuración


In [ ]:
if EXTRACT_DATA_AUDIOS:
    print(len(DATA), len(DATA[0]))
    print(len(LABEL), len(LABEL[0]))

# 5.3. Entrenando preprocesador de datos

In [ ]:
if EXTRACT_DATA_AUDIOS:
    # Normalizando y aplicando PCA
    pca_pipe.fit(DATA)
    pickle.dump(pca_pipe, open(FOLDER_SAVE_NORMALIZADOR_PCA,'wb'))

In [ ]:
if EXTRACT_DATA_AUDIOS:
    DATA_REDU = preparar_datos_para_modelo(DATA)
    print(f"Original vector size: {len(DATA[0])} -> New vector size {len(DATA_REDU[0])} ({int(MINIMA_VARIANA_EXPLICADA*100)}% información mantenida)")

In [ ]:
if EXTRACT_DATA_AUDIOS:
    # Guardando datos
    save_data(DATA_REDU, FOLDER_TRAIN_DATA)
    save_data(LABEL, FOLDER_TRAIN_LABEL)

In [ ]:
spark_data, DATA_REDU_df = read_data(FOLDER_TRAIN_DATA, to_numpy=True)
spark_LABEL, LABEL_df = read_data(FOLDER_TRAIN_LABEL, to_numpy=True)

In [ ]:
if EXTRACT_DATA_AUDIOS:
    print(f"Original vector size: {len(DATA[0])} -> New vector size {len(DATA_REDU_df[0])} ({int(MINIMA_VARIANA_EXPLICADA*100)}% información mantenida)")

# 5.4. Preparando data para train

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(DATA_REDU_df, LABEL_df, test_size=SPLIT_DATA_TRAIN)  # 0.2

# 6. Entrenando el modelo

In [ ]:
#@title Preparando el barrido del modelo para poder hacer GridSearchCV con RandomForestClassifier
if REFIT:
    import multiprocessing

    params_grid = { 
        'n_estimators': [100, 150, 200, 700],
        'max_features': ['auto', 'sqrt', 'log2'],
        'max_depth': [None] + list(range(10, 50)),
        'criterion': ['gini', 'entropy'],
        'min_samples_split': [12, 16, 20]
    }

    rfc = RandomForestClassifier(n_jobs=-1,
                                 max_features= 'sqrt' ,
                                 n_estimators=50, 
                                 verbose=0,
                                 oob_score = True) 

    seed = 1
    grid = GridSearchCV(
            estimator          = rfc,
            param_grid         = params_grid,
            scoring            = 'accuracy',
            n_jobs             = multiprocessing.cpu_count() - 1,
            cv                 = KFold(
                n_splits=10, 
                shuffle=True, 
                random_state=seed),
            refit              = True,
            verbose            = 10,
            return_train_score = True
            )
    
    print("Modelo listo para entrenar")

    @count_elapsed_time
    def train_model(grid_input):
        print("Iniciando entrenamiento del modelo")
        grid_input.fit(X_train, y_train)
        return grid_input

Modelo listo para entrenar


In [ ]:
%%html
function ClickConnect(){ 
    console.log("Clicked on star button"); 
    document.querySelector("iron-icon#star-icon").click() 
} 
setInterval(ClickConnect,10000)

In [ ]:
if REFIT:
    #@title Entrenando
    grid = train_model(grid)

Iniciando entrenamiento del modelo


In [ ]:
if REFIT:
    resultados = pd.DataFrame(grid.cv_results_)
    resultados.filter(regex = '(param*|mean_t|std_t)') \
        .drop(columns = 'params') \
        .sort_values('mean_test_score', ascending = False) \
        .head(4)

In [ ]:
if REFIT:
    # Mejores hiperparámetros por validación cruzada
    # ==============================================================================
    print("----------------------------------------")
    print("Mejores hiperparámetros encontrados (cv)")
    print("----------------------------------------")
    print(grid.best_params_, ":", grid.best_score_, grid.scoring)

In [ ]:
if REFIT:
    # Guardando el modelo
    model = grid.best_estimator_
    pickle.dump(model, open(FOLDER_MODEL, 'wb'))
else:
    # Cargando el modelo guardado
    model = pickle.load(open(FOLDER_MODEL, 'rb'))

In [ ]:
y_final = model.predict(X_valid)

matrix_confusion_personalizada = calcular_porcentajes_aciertos(y_final, LABEL)[2]
print("matrix_confusion_personalizada:", matrix_confusion_personalizada)

print("ACC", metrics.accuracy_score(y_valid, y_final))
print("PREC", metrics.precision_score(y_valid, y_final, average='micro'))

In [ ]:
predicciones = model.predict_proba(X = X_valid)
predicciones[:5, :]

# 7. Testear el modelo

In [ ]:
# Leyendo datos de testeo y calculando vector de caracteristicas
if not os.path.exists(FOLDER_OUT + "new_data/"):
    os.makedirs(FOLDER_OUT + "new_data/")

DATA, LABEL = files_to_data(TEST, save_new_data=str(FOLDER_OUT + "new_data/"))

In [ ]:
DATA_REDU = preparar_datos_para_modelo(DATA)

print(f"Original vector size: {len(DATA[0])} -> New vector size {len(DATA_REDU[0])} ({int(MINIMA_VARIANA_EXPLICADA*100)}% información mantenida)")

In [ ]:
# Guardando datos
save_data(DATA_REDU, FOLDER_TEST_DATA)
save_data(LABEL, FOLDER_TEST_LABEL)

DATA_REDU = read_data(FOLDER_TEST_DATA)
LABEL = read_data(FOLDER_TEST_LABEL)

In [ ]:
y_final = model.predict(DATA_REDU)

matrix_confusion_personalizada = calcular_porcentajes_aciertos(y_final, LABEL)[2]
print("matrix_confusion_personalizada:", matrix_confusion_personalizada)

print("ACC", metrics.accuracy_score(LABEL, y_final))
print("PREC", metrics.precision_score(LABEL, y_final, average='micro'))